<a href="https://colab.research.google.com/github/FinSafe/MachineLearning/blob/main/Regression_Model_for_Predict_%22Total_Household_Income%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Library and Dataset

In [1]:
import pandas as pd
import gdown
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler

import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error

In [2]:
# Helper Function
def import_from_gdrive(id_file, file_name, sheet_name=None):
    link = 'https://drive.google.com/uc?id=' + id_file
    output_file = '/content/' + file_name
    gdown.download(link, output_file, quiet=False)

    df = pd.read_excel(output_file)
    return df

In [3]:
id_file = '1i8urlJHl90Mpq6eVIoOqsfJttx9F1HsA'
df = import_from_gdrive(id_file,'final_dataset_converted_rupiah.xlsx')

Downloading...
From: https://drive.google.com/uc?id=1i8urlJHl90Mpq6eVIoOqsfJttx9F1HsA
To: /content/final_dataset_converted_rupiah.xlsx
100%|██████████| 10.0M/10.0M [00:00<00:00, 55.3MB/s]


In [4]:
df = pd.read_excel('final_dataset_converted_rupiah.xlsx')

In [5]:
fix_columns =['Total Household Income',
              'Household Head Sex',
              'Household Head Age',
              'Household Head Marital Status',
              # 'Household Head Highest Grade Completed',
              'Household Head Highest Grade Completed (Simplified)',
              'Total Household Income',
              # 'Region',
              'Region Category',
              'Type of Building/House',
              'House Floor Area',
              'Number of bedrooms',
              'Electricity',
              'Tenure Status',
              'Type of Household',
              'Total Number of Family members',
              'Total number of family members employed',
              # 'Number of Car, Jeep, Van',
              # 'Number of Motorcycle/Tricycle',
              # 'Number of Television',
              # 'Number of Airconditioner',
              # 'Number of Personal Computer',
              # 'Number of Cellular phone',
              # 'Number of Refrigerator/Freezer',
              # 'Number of Washing Machine',
              'Number of Kids',
              'Number of Vehicles',
              'Number of Communication Devices',
              'Number of Electronics',
              ]

X =  df.loc[:, fix_columns]
X = X.loc[:,~X.columns.duplicated()]

# Mengubah tipe data int menjadi float
int_columns = X.select_dtypes(include=['int32', 'int64']).columns
X[int_columns] = X[int_columns].astype('float64')

y = df['Total Expenses']

In [6]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21930 entries, 0 to 21929
Data columns (total 18 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Total Household Income                               21930 non-null  float64
 1   Household Head Sex                                   21930 non-null  object 
 2   Household Head Age                                   21930 non-null  float64
 3   Household Head Marital Status                        21930 non-null  object 
 4   Household Head Highest Grade Completed (Simplified)  21925 non-null  object 
 5   Region Category                                      21930 non-null  object 
 6   Type of Building/House                               21930 non-null  object 
 7   House Floor Area                                     21930 non-null  float64
 8   Number of bedrooms                                   21930 non-nul

In [7]:
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

# Pipeline preprocessing untuk fitur numerik
numeric_transformer = Pipeline(steps=[
    ('scaler', RobustScaler())  # Standarisasi
])
# Pipeline preprocessing untuk fitur kategorikal
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder())  # One-hot encoding
])

# Gabungkan pipeline preprocessing untuk fitur numerik dan kategorikal
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

X = preprocessor.fit_transform(X)

In [8]:
TEST_SIZE = 0.2
RANDOM_STATE = 42
X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=TEST_SIZE, random_state=RANDOM_STATE)
n_cols = X_train.shape[1]

print("X_train shape : " , X_train.shape)
print("y_train shape : " , y_train.shape)
print("X_test  shape : " , X_test.shape)
print("y_test  shape : " , y_test.shape)

# X_train = X_train.toarray()  # Konversi SparseTensor menjadi numpy array
# X_test = X_test.toarray()    # Konversi SparseTensor menjadi numpy array

X_train shape :  (17544, 45)
y_train shape :  (17544,)
X_test  shape :  (4386, 45)
y_test  shape :  (4386,)


In [9]:
from keras import backend as K
# Fungsi untuk menghitung RMSE
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [10]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.01),input_shape=(X.shape[1],)),
    tf.keras.layers.Dropout(0.2),  # Dropout layer
    tf.keras.layers.Dense(256, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.01)),
    tf.keras.layers.Dropout(0.2),  # Dropout layer
    tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.01)),
    tf.keras.layers.Dropout(0.2),  # Dropout layer
    tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.01)),
    tf.keras.layers.Dropout(0.2),  # Dropout layer
    tf.keras.layers.Dense(1, kernel_initializer='he_normal', activation='linear'),
])

# Kompilasi model

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.mean_absolute_error,
              metrics=["mae"],)

# Early stopping callback
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

# Latih model
model.fit(X_train, y_train, epochs=100, batch_size=128
          ,verbose=2,
          callbacks=[early_stopping])

# Evaluasi model
loss, mae = model.evaluate(X_test, y_test, verbose=0)
print('Mean Squared Error on Test Set:', mae)

# Prediksi
predictions = model.predict(X_test)

# Make predictions
y_preds = model.predict(X_test)

Epoch 1/100
138/138 - 5s - loss: 111687976.0000 - mae: 111687960.0000 - 5s/epoch - 35ms/step
Epoch 2/100
138/138 - 0s - loss: 61279864.0000 - mae: 61279688.0000 - 448ms/epoch - 3ms/step
Epoch 3/100
138/138 - 0s - loss: 23315844.0000 - mae: 23315618.0000 - 446ms/epoch - 3ms/step
Epoch 4/100
138/138 - 0s - loss: 21407360.0000 - mae: 21407132.0000 - 457ms/epoch - 3ms/step
Epoch 5/100
138/138 - 0s - loss: 20216338.0000 - mae: 20216108.0000 - 466ms/epoch - 3ms/step
Epoch 6/100
138/138 - 0s - loss: 19644086.0000 - mae: 19643850.0000 - 467ms/epoch - 3ms/step
Epoch 7/100
138/138 - 0s - loss: 19500018.0000 - mae: 19499776.0000 - 436ms/epoch - 3ms/step
Epoch 8/100
138/138 - 0s - loss: 19283588.0000 - mae: 19283346.0000 - 433ms/epoch - 3ms/step
Epoch 9/100
138/138 - 0s - loss: 19097478.0000 - mae: 19097236.0000 - 442ms/epoch - 3ms/step
Epoch 10/100
138/138 - 0s - loss: 19092216.0000 - mae: 19091970.0000 - 449ms/epoch - 3ms/step
Epoch 11/100
138/138 - 0s - loss: 19302910.0000 - mae: 19302670.0000 

# Result

In [12]:
# Epoch 44/100
# 138/138 - 1s - loss: 19111542.0000 - mae: 19111300.0000 - 669ms/epoch - 5ms/step
# Mean Squared Error on Test Set: 17637802.0

# Save Model

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               11776     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 128)               1

In [14]:
model.save('model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
model.save('saved_model/')